# W207 Final Project: Facial Recognition

## Introduction

This is the final project submission from Team MapReduce,MapReuse,MapRecycle for W207: Applied Machine Learning as part of the MIDS program at UC Berkeley. We are working with data from https://www.kaggle.com/c/facial-keypoints-detection/data and the goal is to accurately identify facial features on an (x, y) coordinate system based on an input image. 

## Communicate on SLACK when working on main notebook as much as possible

*Data Observations*
- There are half face images, etc

*Ideas to try*
- Use other features to predict one specific feature

- Break out the image into its own matrix where each pixel is a feature and then predict each facial characteristic position
    - First use subset of images that contain all key points and then just predict the locations. 

*Feature Improvement Ideas*
- Rotating the images a bit
- Blurring the images a bit
- Maybe normally the pixel values by average pixel shade of every picture
- Seperate the image into different quadrants 
- Compressing 256 base image to 16 base image. Just black and white? Interval of -1 to 1?

*Knockout List/Plan*
- Print first 10-20 images generally. 
- Print edge cases for different facial feature locations.

- Very basic test with a couple different basic algorithms. 
- Start with very basic one hidden layer perception
- Improve neural network to be CNN architecture. 
- Prepare a summary table of quality metrics: L2 distance, L1 distance

*Naming/Working Rules*
- train_data, train_labels
- dev_data, dev_labels

- Feature Engineering: put "_transformation done". Ex: train_data_blur

*Useful Items*
- Learn Keras
- https://www.kaggle.com/c/facial-keypoints-detection#getting-started-with-r
- http://adventuresinmachinelearning.com/keras-tutorial-cnn-11-lines/
- https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm#k-NN_regression

## Data

In [1]:
# Import a bunch of libraries.
import time
import os
# os.environ["MKL_THREADING_LAYER"] = "GNU"
import keras
import theano
import numpy as np
import pandas as pd
import decimal
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from scipy import stats

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

print 'theano version:', theano.__version__
print 'theano config device:', theano.config.device, theano.config.floatX
print 'keras version:', keras.__version__
print 'pandas version:', pd.__version__
print 'numpy version:', np.__version__

Using Theano backend.



You can find the C code in this temporary file: c:\users\user\appdata\local\temp\theano_compilation_error_srdxro


Exception: Compilation failed (return status=1): C:\Users\User\AppData\Local\Temp\cc2Qh0ql.o: In function `unpack_list_of_ssize_t':. C:/Users/User/AppData/Local/Theano/compiledir_Windows-10-10.0.17134-AMD64_Family_21_Model_96_Stepping_1_AuthenticAMD-2.7.15-64/lazylinker_ext/mod.cpp:48: undefined reference to `__imp_PyExc_TypeError'. C:/Users/User/AppData/Local/Theano/compiledir_Windows-10-10.0.17134-AMD64_Family_21_Model_96_Stepping_1_AuthenticAMD-2.7.15-64/lazylinker_ext/mod.cpp:58: undefined reference to `__imp_PyExc_IndexError'. C:\Users\User\AppData\Local\Temp\cc2Qh0ql.o: In function `CLazyLinker_init':. C:/Users/User/AppData/Local/Theano/compiledir_Windows-10-10.0.17134-AMD64_Family_21_Model_96_Stepping_1_AuthenticAMD-2.7.15-64/lazylinker_ext/mod.cpp:331: undefined reference to `__imp_PyCObject_Type'. C:/Users/User/AppData/Local/Theano/compiledir_Windows-10-10.0.17134-AMD64_Family_21_Model_96_Stepping_1_AuthenticAMD-2.7.15-64/lazylinker_ext/mod.cpp:352: undefined reference to

In [55]:
# Load the data from downloaded file and split the train into a smaller train and dev

train = pd.read_csv('training.csv')
test = pd.read_csv('test.csv')

print "Columns available:"
print train.columns.values 

def process_datasets(data):
    facial_point_labels = data.iloc[:, :-1]
    images_pixel_feature = data.Image.apply(lambda im: np.fromstring(im, sep=' '))
    
    # Turn below on to the regressions below.
    #images_pixel_feature = pd.DataFrame(images_pixel_feature.values.tolist())
    
    # Filling NAs with 0 
    facial_point_labels = facial_point_labels.fillna(0)
    images_pixel_feature = images_pixel_feature.fillna(0)
    
    return images_pixel_feature, facial_point_labels

train_all_features, train_all_labels = process_datasets(train)

percent_to_dev = 0.2 # 20% of the data will go to dev

train_data, dev_data, train_labels, dev_labels = train_test_split(train_all_features, train_all_labels, test_size=percent_to_dev)
test_data, test_labels = process_datasets(test)

print "\nTrain data shape", train_data.shape
print "Dev data shape", dev_data.shape
print "Train labels shape", train_labels.shape
print "Dev labels shape", dev_labels.shape

print "\nData Generation Finished"

Columns available:
['left_eye_center_x' 'left_eye_center_y' 'right_eye_center_x'
 'right_eye_center_y' 'left_eye_inner_corner_x' 'left_eye_inner_corner_y'
 'left_eye_outer_corner_x' 'left_eye_outer_corner_y'
 'right_eye_inner_corner_x' 'right_eye_inner_corner_y'
 'right_eye_outer_corner_x' 'right_eye_outer_corner_y'
 'left_eyebrow_inner_end_x' 'left_eyebrow_inner_end_y'
 'left_eyebrow_outer_end_x' 'left_eyebrow_outer_end_y'
 'right_eyebrow_inner_end_x' 'right_eyebrow_inner_end_y'
 'right_eyebrow_outer_end_x' 'right_eyebrow_outer_end_y' 'nose_tip_x'
 'nose_tip_y' 'mouth_left_corner_x' 'mouth_left_corner_y'
 'mouth_right_corner_x' 'mouth_right_corner_y' 'mouth_center_top_lip_x'
 'mouth_center_top_lip_y' 'mouth_center_bottom_lip_x'
 'mouth_center_bottom_lip_y' 'Image']

Train data shape (5639L,)
Dev data shape (1410L,)
Train labels shape (5639, 30)
Dev labels shape (1410, 30)

Data Generation Finished


In [52]:
# Print first couple labels
train_labels.iloc[:5,:5]

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x
448,69.009155,37.392891,31.856165,35.623621,63.259052
686,66.104842,34.633263,29.693684,35.197895,59.330526
4236,67.640071,35.673855,31.773689,36.314336,0.000000
204,66.902495,37.388198,30.371168,39.087303,60.956198
2821,68.292978,37.460391,29.048711,38.626062,0.000000


In [ ]:
# EDA: Print a few faces (Mark)

for example in range(2,5):
    img = train_data[example].reshape(96,96)
    img.shape
    
    plt.figure()
    plt.imshow(img, cmap='gray')
    for c in range(0, len(train_labels.columns)-1, 2):
        x = train_labels[train_labels.columns[c]][example]
        y = train_labels[train_labels.columns[c+1]][example]
        plt.scatter(x,y,c='r')

In [ ]:
# EDA: Print interesting edge cases images (Mark)

# NOTES: Currently having plotting issues. See issue #2 on GitHub for more information.

interesting_cases = []

# additional interesting cases to explore if time allows:
# - multiple faces in one image
# - upside down faces
# - incomplete faces (not all key points present)
# - race/lighting (average shade of pixels in facial region, plot couple darkest/lightest)
# - faces halfway offscreen/covered (one eye present, one not)

# compile interesting cases
interesting_cases.extend(train_labels.nose_tip_x.nsmallest(2).keys())          # noses near left of screen
interesting_cases.extend(train_labels.nose_tip_x.nlargest(2).keys())           # noses near right of screen
interesting_cases.extend(train_labels.nose_tip_y.nsmallest(2).keys())          # noses near top of screen
interesting_cases.extend((train_labels.right_eye_center_x -
                          train_labels.left_eye_center_x).nsmallest(2).keys()) # small faces
interesting_cases.extend((train_labels.right_eye_center_x -
                          train_labels.left_eye_center_x).nlargest(2).keys())  # wide faces

# plot these cases with key points
for index in interesting_cases:
    train_data_square = train_data[index].reshape(96,96)
    plt.figure()
    plt.imshow(train_data_square, cmap='gray')
    for c in range(0, len(train_labels.columns)-1, 2):
        x = train_labels[train_labels.columns[c]][index]
        y = train_labels[train_labels.columns[c+1]][index]
        plt.scatter(x,y,c='r')

In [ ]:
# Algorithms to try:
# Need to activate transformation above to change list of items to actual columns to get it working. Is this right way to do it?

regressors = {
    'LinearRegression': LinearRegression(),
    'DecisionTree': DecisionTreeRegressor(),
    'NearestNeighbors': KNeighborsRegressor()
} 

def model_training(model):
    
    X = train_data
    Y = train_labels['left_eye_center_x']
        
    model.fit(X,Y)
    predictions = model.predict(dev_data)
    
    mse = mean_squared_error(dev_labels['left_eye_center_x'], predictions)
    
    return mse

for k,v in regressors.items():
    print k,':',model_training(v)

In [7]:
# Try Simple Single Hidden Layer (800 node) Neural Net Model (Madeline, Alex):

"""
"""

'\n'

In [1]:
# Build more complex CNN model (Madeline, Alex)

### PSEUDOCODE ###
def build_model():
    
    # start with the keras Sequential model which is just a stack of 
    # layers
    model = keras.Sequential()
    
    # add a 2D convolutional layer
    model.add_convolutional_2D_layer()
    
    # add a 2D pooling layer
    model.add_Pooling_Layer()
    
    # add another 2D convolutional layer
    model.model.add_convolutional_2D_layer()
    
    # add another pooling layer
    model.add_Pooling_Layer()
    
    # add output layer using softmax activation
    model.add_output_layer()
    
    return model

In [ ]:
# Train the model

### PSEUDOCODE ###

model.compile()
model.fit(train_data, train_labels)

In [ ]:
# Make predictions on the test data

### PSEUDOCODE ###
pred = model.predict(test_data)

In [ ]:
# print out some examples of the predictions

### PSEUDOCODE ###
fig = plt.figure(figsize=12,8)
idx = np.random.randint(0, len(test_data), size=16)
for i in idx:
    img = test_data[i].reshape(96, 96)
    ax = fig.add_subplot(4, 4, i + 1, xticks=[], yticks=[])
    ax.imshow(img, cmap='gray')
    for c in range(0, len(pred.columns)-1, 2):
        x_pred = pred[pred.columns[c]][0]
        y_pred = pred[pred.columns[c+1]][0]
        ax.scatter(x_pred, y_pred, c='r')